In [29]:
import cv2
import numpy as np
from ultralytics import YOLO
import pytesseract
import os
import time
import csv


In [30]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [31]:
save_folder = "D:/anpr2/cropped_plates/"
os.makedirs(save_folder, exist_ok=True)

In [32]:
csv_file = "D:/anpr2/number_plate_data.csv"
csv_header = ['Frame', 'Plate Number', 'X1', 'Y1', 'X2', 'Y2', 'Saved Image Path']


if not os.path.exists(csv_file):
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_header)

In [33]:
yolo_model = YOLO("D:/anpr2/runs/detect/train_final.pt")

In [34]:
cap = cv2.VideoCapture(0)
frame_count = 0
fps = 0

In [35]:

while True:
    start_time = time.time()  
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    frame_count += 1
    if frame_count % 5 != 0:  
        continue

    results = yolo_model(frame)
    
    print(f"YOLO detected {len(results[0].boxes)} objects")

    detected_boxes = results[0].boxes
    if len(detected_boxes) == 0:
        print("No number plates detected")
        continue  

    for result in detected_boxes:
        confidence = result.conf[0].item()  
        if confidence > 0.5:  
            
            x1, y1, x2, y2 = map(int, result.xyxy[0])  
            number_plate_image = frame[y1:y2, x1:x2]  

            if number_plate_image.size == 0:
                print("Empty number plate image detected, skipping OCR...")
                continue

            number_plate_image = cv2.cvtColor(number_plate_image, cv2.COLOR_BGR2GRAY)
            number_plate_image = cv2.medianBlur(number_plate_image, 3)  

            plate_number = pytesseract.image_to_string(number_plate_image, config='--psm 8')  

            plate_number = ''.join(e for e in plate_number if e.isalnum()).strip()

            if not plate_number:
                print("OCR failed to detect text.")
                continue

            cv2.putText(frame, "Number Plate Detected", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            lp_filename = f"{save_folder}number_plate_{frame_count}_{x1}_{y1}.jpg"
            cv2.imwrite(lp_filename, frame[y1:y2, x1:x2])  

            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([frame_count, plate_number, x1, y1, x2, y2, lp_filename])

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  
            cv2.imshow("Cropped Number Plate", frame[y1:y2, x1:x2])

    cv2.imshow("Number Plate Recognition", frame)

    fps = 1 / (time.time() - start_time)
    print(f"FPS: {fps:.2f}")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 13.4ms
Speed: 2.6ms preprocess, 13.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No number plates detected

0: 480x640 (no detections), 13.8ms
Speed: 1.3ms preprocess, 13.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No number plates detected

0: 480x640 (no detections), 8.4ms
Speed: 1.9ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No number plates detected

0: 480x640 (no detections), 9.7ms
Speed: 1.0ms preprocess, 9.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No number plates detected

0: 480x640 (no detections), 9.4ms
Speed: 1.0ms preprocess, 9.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
YOLO detected 0 objects
No number plates detected

0: 480x640 (no detections), 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 0.0ms postpr